In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import wandb
import os
import torch.optim as optimizers
import dfs_code
from torch_geometric.data import InMemoryDataset, Data
import pickle
import torch
import torch.nn as nn
import tqdm
import copy
import pandas as pd
import torch.nn.functional as F
import sys
import yaml
import functools
from ml_collections import ConfigDict
sys.path = ['../../src'] + sys.path
from dfs_transformer import collate_dgl, Gilmer, Trainer, to_cuda, Enzymes
from graphein.protein.resi_atoms import RESI_THREE_TO_1, AMINO_ACIDS


2021-10-05 15:14:17.330244: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt/lib:/opt/intel/tbb/lib:/opt/intel/clck/2019.0/lib:/opt/intel/compilers_and_libraries_2019/linux/lib:/opt/intel/compilers_and_libraries/linux/lib:/opt/intel/itac/2019.0.018/lib:/opt/intel/itac_2019/intel64/lib:/opt/intel/itac_latest/intel64/lib:/opt/intel/parallel_studio_xe_2019.0.045/clck_2019/lib:/opt/intel/parallel_studio_xe_2019.0.045/itac_2019/l

To use the Graphein submodule graphein.protein.features.sequence.embeddings, you need to install biovec.

To do so, use the following command:

    pip install biovec
To use the Graphein submodule graphein.protein.visualisation, you need to install pytorch3d.

pytorch3d cannot be installed via pip
To use the Graphein submodule graphein.protein.meshes, you need to install pytorch3d.

pytorch3d cannot be installed via pip


In [3]:
collate_fn = functools.partial(collate_dgl, node_feats="node_features", edge_feats="edge_features", target="y")

In [4]:
m = ConfigDict()
t = ConfigDict()
d = ConfigDict()

In [5]:
m["class"] = "Gilmer"
m["n_tasks"] = 1
m["node_out_feats"] = 64
m["edge_hidden_feats"] = 128
m["num_step_message_passing"] = 3
m["num_step_set2set"] = 6
m["num_layer_set2set"] = 3
m["mode"] = 'classification'
m["number_atom_features"] = 26
m["number_bond_features"] = 8
m["n_classes"] = 384
m["nfeat_name"] = 'x'
m["efeat_name"] = 'edge_attr'


t["batch_size"] = 50
t["gpu_id"] = None
t["load_last"] = False
t["fingerprint"] = "cls"
t["accumulate_grads"] = 2
t["alpha"] = 0.0
t["clip_gradient"] = 0.5
t["decay_factor"] = 0.8
t["es_improvement"] = 0.0
t["es_path"] = None
t["es_patience"] = 30
t["es_period"] = 203
t["lr"] = 0.003
t["lr_patience"] = 3
t["lr_adjustment_period"] = 203
t["wdecay_encoder"] = 0.0
t["n_epochs"] = 1000
t["seed"] = 123
t["num_workers"] = 8

d["n_classes"] = 384
d["path"] = "/mnt/ssd/datasets/enzyme/graphein_basic_n1000_m500.pkl"
d["n_edge_types"] = 8
d["n_node_types"] = 26

In [6]:
mode = "online"
name = "gilmer"
project = "enzymes-n200"

In [7]:
dataset = Enzymes(path=d.path, acids2int=AMINO_ACIDS)

In [8]:
train_idx = torch.tensor([idx for idx, d in enumerate(dataset) if d.split == "train"], dtype=torch.long)
valid_idx = torch.tensor([idx for idx, d in enumerate(dataset) if d.split == "valid"], dtype=torch.long)
test_idx = torch.tensor([idx for idx, d in enumerate(dataset) if d.split == "test"], dtype=torch.long)

In [9]:
coll_train = collate_fn

In [10]:
trainloader = DataLoader(dataset, sampler=torch.utils.data.SubsetRandomSampler(train_idx), 
                         batch_size=t.batch_size, collate_fn=coll_train, num_workers=t.num_workers)
validloader = DataLoader(dataset, sampler=torch.utils.data.SubsetRandomSampler(valid_idx), 
                         batch_size=t.batch_size, collate_fn=collate_fn, num_workers=t.num_workers)
testloader = DataLoader(dataset, sampler=torch.utils.data.SubsetRandomSampler(test_idx), 
                        batch_size=t.batch_size, collate_fn=collate_fn, num_workers=t.num_workers)

In [11]:
data = next(iter(trainloader))

In [12]:
data

(Graph(num_nodes=8347, num_edges=17396,
       ndata_schemes={'x': Scheme(shape=(26,), dtype=torch.float32)}
       edata_schemes={'edge_attr': Scheme(shape=(8,), dtype=torch.float32)}),
 tensor([357, 162,  29, 162, 351, 178, 155, 351, 376, 162,  65, 369, 198,  25,
         162, 157, 330, 233, 243, 109, 190, 271, 240, 218,  62,  65,  65,  47,
         247,  62, 190, 372, 337,  85, 166, 190, 351, 341, 351, 162, 118, 266,
         155, 282,  84, 361,  62, 351,  84, 277]))

In [13]:
config = ConfigDict()
config["model"] = m
config["training"] = t
config["data"] = d

In [14]:
run = wandb.init(mode=mode, project=project, entity="dfstransformer", 
                 name=name, config=config.to_dict(), job_type="evaluation")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chrisxx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-10-05 15:14:30.029528: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt

In [15]:
ce = nn.CrossEntropyLoss(ignore_index=-1)

In [16]:
def loss(pred, y, ce=ce):
    return ce(pred, y)

def acc(pred, y):
    return torch.sum(torch.argmax(pred, dim=1) == y)/len(y)

In [17]:
#device = torch.device('cuda:%d'%t.gpu_id if torch.cuda.is_available()  else 'cpu')
device = 'cpu'

In [18]:
model = Gilmer(**m)

In [19]:
t

accumulate_grads: 2
alpha: 0.0
batch_size: 50
clip_gradient: 0.5
decay_factor: 0.8
es_improvement: 0.0
es_path: null
es_patience: 30
es_period: 203
fingerprint: cls
gpu_id: null
load_last: false
lr: 0.003
lr_adjustment_period: 203
lr_patience: 3
n_epochs: 1000
num_workers: 8
seed: 123
wdecay_encoder: 0.0

In [20]:
trainer = Trainer(model, trainloader, loss, validloader=validloader, metrics={'acc': acc}, es_argument=lambda log: -log['valid-acc'], wandb_run = run, **t)

In [ ]:
trainer.fit()

Epoch 1: loss 5.605309 0.0800:  10%|██████████████▋                                                                                                                               | 21/203 [00:26<03:45,  1.24s/it]

In [ ]:
model.load_state_dict(torch.load(trainer.es_path+'checkpoint.pt'))

In [ ]:
def compute_acc(model, loader):
    with torch.no_grad():
        preds = []
        ys = []
        for data in tqdm.tqdm(loader):
            data = [to_cuda(dd, device) for dd in data]
            pred = model(*data[:-1])
            pred = torch.argmax(pred, dim=1).detach().cpu().numpy().tolist()
            y = data[-1].detach().cpu().numpy().tolist()
            preds += pred
            ys += y
    return (np.asarray(preds) == np.asarray(ys)).sum()/len(ys)
            

In [ ]:
run.log({'Valid Accuracy': compute_acc(model, validloader)})
run.log({'Test Accuracy': compute_acc(model, testloader)})

In [ ]:
compute_acc(model, testloader)

In [ ]:
compute_acc(model, validloader)